### Podstawy Sparka
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### <center>Cześć 1</center>

In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('Czesc 1 BigData').getOrCreate()

print(spark.version)

In [ ]:
data = [i + 1 for i in range(5)]

rdd = spark.sparkContext.parallelize(data)

print('RDD:', rdd.collect())

In [ ]:
df = spark.createDataFrame([('Andorra', 98), ('United Arab Emirates', 1683)], ['country_name', 'total_confirmed'])

df.show()

In [ ]:
spark.stop()

### <center>Część 2</center>

### 2.1. Wczytaj dane ze swojego pliku CSV dotyczącego danych COVID-19 (pamiętaj o nagłówku dla kolumn).

In [ ]:
import pandas as pd
import os


covid_df = pd.read_csv('data/covid.csv')

def save_spark_df(df, name):
    if 'result' not in os.listdir(os.getcwd()):
        os.mkdir('result')
    df.write.csv(f'{name}.csv', header=True)
    df.write.json(f'{name}.json')
    df.write.parquet(f'{name}.parquet')
    df.write.format('avro').save(f'{name}.avro')

### 2.3. Wyświetl co najmniej 5 kolumn i wyjaśnij ich znaczenie dla analizy przypadków COVID-19. 

In [ ]:
covid_df = covid_df[['date', 'country_name', 'Continent', 'age_confirmed_0', 'age_confirmed_1', 'age_deceased_0', 'age_deceased_1', 'health_expenditure_usd', 'life_expectancy', 'school_closing']]

### 2.2. Utwórz DataFrame. Pamiętaj o utworzeniu własnego schematu z wybranymi kolumnami. 

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType


spark = SparkSession.builder.appName('Czesc 2 BigData').getOrCreate()

schema = StructType([
    StructField('date', StringType(), True),
    StructField('country_name', StringType(), True),
    StructField('continent', StringType(), True),
    StructField('preschoolers_confirmed', IntegerType(), True),
    StructField('pupils_confirmed', IntegerType(), True),
    StructField('preschoolers_dead', IntegerType(), True),
    StructField('pupils_dead', IntegerType(), True),
    StructField('health_expenditure_usd', FloatType(), True),
    StructField('life_expectancy', FloatType(), True),
    StructField('school_closing', IntegerType(), True),
])

spark_covid_df = spark.createDataFrame(covid_df, schema)

### 2.3. Wyświetl co najmniej 5 kolumn i wyjaśnij ich znaczenie dla analizy przypadków COVID-19.

In [ ]:
save_spark_df(spark_covid_df, name='2_3')
spark_covid_df.show(5)

### 2.4. Dodaj kolumnę, która będzie zbierała dane, które uznasz za stosowne i które są istotne z punktu widzenia analizy COVID-19 (np. wyliczenie gęstości zaludnienia na podstawie liczby ludności i powierzchni kraju).

In [ ]:
from pyspark.sql.functions import concat, col


spark_covid_df = spark_covid_df.withColumn(
    'who_died_more', 
    (col('preschoolers_dead') - col('pupils_dead')) * 100
).withColumn(
    'who_ill_more', 
    (col('preschoolers_confirmed') - col('pupils_confirmed')) * 100
)

save_spark_df(spark_covid_df, name='2_4')
spark_covid_df.show(5)

### 2.5. Połącz wybrane kolumny ze sobą w sposób niosący nowe informacje za pomocą funkcji concat .

In [ ]:
spark_covid_df = spark_covid_df.withColumn(
    'combined_country_continent',
    concat(col('country_name'), col('continent')),
)

save_spark_df(spark_covid_df, name='2_5')
spark_covid_df.show(5)

### 2.6. Wyfiltruj co najmniej 10 wybranych przez siebie, istotnych informacji (np. kraj z liczbą łóżek większą od 10000 czy dzień, w którym było najmniej zachorowań w danym kraju).

In [ ]:
filtered_df = spark_covid_df.filter((col('pupils_confirmed') > 100) & (col('preschoolers_dead') > 100))

save_spark_df(filtered_df, name='2_6')
filtered_df.show(5)

### 2.7. Wykonaj 5 agregacji na swoim DF (np. min, max, agg, count, mean) uprzednio pogrupowawszy kolumny (groupBy).

In [ ]:
aggregated_df = spark_covid_df.groupBy("country_name").agg(
    {"preschoolers_confirmed": "max",
     "pupils_confirmed": "sum",
     "preschoolers_dead": "avg",
     "pupils_dead": "min",
     "health_expenditure_usd": "mean"}
)

save_spark_df(aggregated_df, name='2_7')
aggregated_df.show(5)